In [1]:
import numpy as np
import scipy as sp
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.colors import LogNorm
import scipy.stats as stats
from matplotlib import gridspec
import aplpy as apl #The Astronomy Plotting Library for python
import astropy
from astropy.stats import sigma_clip
from astropy.modeling import functional_models, models, fitting
import astropy.units as U
from astropy.coordinates import ICRS, Galactic, FK4, FK5, Angle, Latitude, Longitude
import astropy.constants as C
from astropy import wcs
import astropy.io.fits as fits
from astropy.io import ascii
import astropy.coordinates as coord
from astropy.coordinates import SkyCoord
from astropy.stats import signal_to_noise_oir_ccd as SNRas
#import spectral_cube as SC
#from spectral_cube import SpectralCube as sc
from astropy.wcs import WCS
matplotlib.use('Agg')
import matplotlib.cm as cm
#import astrometry as ast
import pyfits
import requests
import json
import os
from sklearn import datasets, linear_model
from scipy.optimize import curve_fit
import numpy.ma as ma
from astropy.table import Table, Column
import pyextract
import sewpy
import pandas as pd
from astropy import units as u

/home/lynge/anaconda2/lib/python2.7/site-packages/matplotlib/__init__.py:1405: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


In [2]:


k_g = 0.16
k_r = 0.08
k_i = 0.02
k_z = 0.01

PG_RA = 201.4144438
PG_DEC = -8.8219767

RA_SA100_241 = 133.14186958
DEC_SA100_241 = -0.66360306

RA_SA101_315 = 148.71371625  
DEC_SA101_315 = -0.45861306

P_mag_g = 12.225
P_mag_r = 11.918
P_mag_i = 11.829
P_mag_z = 11.814

G_mag_g = 13.159
G_mag_r = 12.437
G_mag_i = 12.123
G_mag_z = 11.944

T_mag_g = 12.215
T_mag_r = 11.850
T_mag_i = 11.720
T_mag_z = 11.675


'''
g_SA100_241 = 10.166
r_SA100_241 = 10.229
i_SA100_241 = 10.347
z_SA100_241 = 10.456

g_SA101_315 = 11.703
r_SA101_315 = 10.862
i_SA101_315 = 10.560
z_SA101_315 = 10.416
'''

sta1_RA_WORLD = SkyCoord('22:08:51.993  -47:10:28.64', unit=(u.hourangle, u.deg)).ra.degree #UCAC3 86-422902
sta1_DEC_WORLD = SkyCoord('22:08:51.993  -47:10:28.64', unit=(u.hourangle, u.deg)).dec.degree

sta2_RA_WORLD = SkyCoord('22:08:51.908  -47:07:20.88', unit=(u.hourangle, u.deg)).ra.degree #TYC 8438-1960-1
sta2_DEC_WORLD = SkyCoord('22:08:51.908  -47:07:20.88', unit=(u.hourangle, u.deg)).dec.degree

G_RA = SkyCoord('12:41:41 -00:26:26', unit=(u.hourangle, u.deg)).ra.degree #Random patch of sky
G_DEC = SkyCoord('12:41:41 -00:26:26', unit=(u.hourangle, u.deg)).dec.degree 

T_RA = SkyCoord('15:30:39 +06:01:13', unit=(u.hourangle, u.deg)).ra.degree #Random patch of sky
T_DEC = SkyCoord('15:30:39 +06:01:13', unit=(u.hourangle, u.deg)).dec.degree 


In [3]:
NGC3783_g = [os.path.join('/media/lynge/Lynge-Back-up/NGC7213-cal/21-06-17/NGC7213-g',f)
             for f in os.listdir('/media/lynge/Lynge-Back-up/NGC7213-cal/21-06-17/NGC7213-g')]

G_g = [os.path.join('/media/lynge/Lynge-Back-up/NGC7213-cal/21-06-17/G104-426-g/',f) \
        for f in os.listdir('/media/lynge/Lynge-Back-up/NGC7213-cal/21-06-17/G104-426-g/')]

T_g = [os.path.join('/media/lynge/Lynge-Back-up/NGC7213-cal/21-06-17/TYC-362-266-1-g/',f) \
        for f in os.listdir('/media/lynge/Lynge-Back-up/NGC7213-cal/21-06-17/TYC-362-266-1-g/')]

for i in range(len(T_g)):
    print pyfits.getdata(T_g[i])[300,300]

6.15573770492
-1.25833333333
13.1475409836
-10.479338843
-6.88524590164
12.8429752066
-3.18181818182
-6.82113821138


/home/lynge/anaconda2/lib/python2.7/site-packages/pyfits/hdu/image.py:515: VerifyWarning: Invalid 'BLANK' keyword in header.  The 'BLANK' keyword is only applicable to integer data, and will be ignored in this HDU.
  warnings.warn(msg, VerifyWarning)


In [4]:
NGC3783_r = [os.path.join('/media/lynge/Lynge-Back-up/NGC7213-cal/21-06-17/NGC7213-r',f)
             for f in os.listdir('/media/lynge/Lynge-Back-up/NGC7213-cal/21-06-17/NGC7213-r')]

G_r = [os.path.join('/media/lynge/Lynge-Back-up/NGC7213-cal/21-06-17/G104-426-r/',f) \
        for f in os.listdir('/media/lynge/Lynge-Back-up/NGC7213-cal/21-06-17/G104-426-r/')]

T_r = [os.path.join('/media/lynge/Lynge-Back-up/NGC7213-cal/21-06-17/TYC-362-266-1-r/',f) \
        for f in os.listdir('/media/lynge/Lynge-Back-up/NGC7213-cal/21-06-17/TYC-362-266-1-r/')]

In [5]:
NGC3783_i = [os.path.join('/media/lynge/Lynge-Back-up/NGC7213-cal/21-06-17/NGC7213-i',f)
             for f in os.listdir('/media/lynge/Lynge-Back-up/NGC7213-cal/21-06-17/NGC7213-i')]

G_i = [os.path.join('/media/lynge/Lynge-Back-up/NGC7213-cal/21-06-17/G104-426-i/',f) \
        for f in os.listdir('/media/lynge/Lynge-Back-up/NGC7213-cal/21-06-17/G104-426-i/')]

T_i = [os.path.join('/media/lynge/Lynge-Back-up/NGC7213-cal/21-06-17/TYC-362-266-1-i/',f) \
        for f in os.listdir('/media/lynge/Lynge-Back-up/NGC7213-cal/21-06-17/TYC-362-266-1-i/')]

In [6]:
NGC3783_z = [os.path.join('/media/lynge/Lynge-Back-up/NGC7213-cal/21-06-17/NGC7213-z',f)
             for f in os.listdir('/media/lynge/Lynge-Back-up/NGC7213-cal/21-06-17/NGC7213-z')]

G_z = [os.path.join('/media/lynge/Lynge-Back-up/NGC7213-cal/21-06-17/G104-426-z/',f) \
        for f in os.listdir('/media/lynge/Lynge-Back-up/NGC7213-cal/21-06-17/G104-426-z/')]

T_z = [os.path.join('/media/lynge/Lynge-Back-up/NGC7213-cal/21-06-17/TYC-362-266-1-z/',f) \
        for f in os.listdir('/media/lynge/Lynge-Back-up/NGC7213-cal/21-06-17/TYC-362-266-1-z/')]

In [7]:
def dist(mid,end):
    return np.sqrt((mid[0] - end[0])**2 + (mid[1] - end[1])**2)

In [8]:
def flux_cal(A_mag,B_mag,B_flux):
    '''Flux of A found through magnitude and flux of B'''
    return B_flux*10**((B_mag-A_mag)/2.5)

In [9]:
def Basic(quasar,header):
    '''Provides a basic list of neccesities from the fits header file for use in the later \
    flux determination, it does not serve an individual purpose beyound that'''
    AGNdata = quasar #np.rot90(np.rot90(np.fliplr(quasar)))
    #header = data[0].header
    #print header
    RAstd = header['CRVAL1']
    DECstd = header['CRVAL2']
    #print RAstd, DECstd
    pixspa = header['PIXSCALE']/(3600.)
    #print pixspa
    RAstdpix = header['CRPIX1']
    DECstdpix = header['CRPIX2']
    exptime = 10 # header['EXPTIME']
    return header,AGNdata,RAstd,DECstd,pixspa,RAstdpix,DECstdpix,exptime

In [10]:
def curve(quasar,header,w,center,apparature):
    '''Determines the FLUX of the stellar object given the the numpy array of the image, \
    the header of the fits file, the astropy coordinate representation, \
    the object position and the apparature of interest'''
    header,AGNdata,RAstd,DECstd,pixspa,RAstdpix,DECstdpix,exptime = Basic(quasar,header)
    #AGNdata = np.swapaxes(AGNdata,0,1)
    y, x = np.ogrid[0:header['NAXIS1'],0:header['NAXIS1']]
    #w = np.swapaxes(w,0,1)
    #F_AGN = np.zeros((512,512,3)) #((int(2*apparature/pixspa)+1,int(2*apparature/pixspa)+1,3))
    #print center
    x1 = w.all_world2pix(center[0],center[1],0)
    x1,y1 = (x1[0]),(x1[1])
    try:
        mask = ((y-y1)**2 + (x-x1)**2) > (apparature/float(pixspa))**2
        AGNdata[mask] = float(0)
        mask = AGNdata == 0.
        rows = np.flatnonzero((~mask).sum(axis=1))
        cols = np.flatnonzero((~mask).sum(axis=0))
        AGNdata = AGNdata[rows.min():rows.max()+1, cols.min():cols.max()+1]
        #print apparature/float(pixspa)
    except:
        AGNdata = np.array([[float('nan'),float('nan')],[float('nan'),float('nan')]])
    #if np.sum(AGNdata) < 500:
    #    AGNdata = np.array([[float('nan'),float('nan')],[float('nan'),float('nan')]])
    return AGNdata/float(exptime), x1, y1


In [11]:
def comp1(qua,sta):
    '''Code to determine the relation between the AGN and the Stellar flux as determined by either me \
    or by the SExtractor program respectively'''
    comp = qua/sta
    return comp

In [12]:
def SNR(data1,data2,header,ap1,ap2):
    npix1 = np.pi*ap1**2/((header['PIXSCALE']/3600.)**2)
    npix2 = np.pi*ap2**2/((header['PIXSCALE']/3600.)**2)
    #data2_1 = ((data2-data1)/npix2)*npix1
    data1_2 = data1/10.
    data2_2 = data2/10.
    return SNRas(10.,data1_2,abs(data2_2),0,header['RON'],npix1,gain=header['GAIN'])

In [13]:
def SXtable(quas,aperture):
    header = pyfits.getheader(quas)
    phot_app = aperture*3600/header['PIXSCALE']
    np.set_printoptions(threshold=np.inf)
    sew = sewpy.SEW(params=['XMODEL_WORLD', 'YMODEL_WORLD', 'X2MODEL_WORLD', 'Y2MODEL_WORLD', 
                            'XMODEL_IMAGE', 'YMODEL_IMAGE', 'FLUX_RADIUS(1)', 'FLUX_APER','ELLIPTICITY'],
                    config={"DETECT_THRESH":10, 'DETECT_MINAREA':20, 'DETECT_MAXAREA':30000, 
                            "PHOT_FLUXFRAC":"0.5", 'PHOT_APERTURES':2*phot_app, 
                            'PIXEL_SCALE':header['PIXSCALE']}
                    , sexpath='/usr/bin/sextractor')
    out = sew(quas)
    return out

In [14]:
def flux(quas,aperture,main_RA_WORLD,main_DEC_WORLD):
    header = pyfits.getheader(quas)
    RAmain = WCS(quas).all_world2pix(main_RA_WORLD,main_DEC_WORLD,1)[0]
    DECmain = WCS(quas).all_world2pix(main_RA_WORLD,main_DEC_WORLD,1)[1]
    output = float('nan')
    if 15 < RAmain < header['NAXIS1'] - 15 and 15 < DECmain < header['NAXIS2'] - 15: 
        out = SXtable(quas,aperture)
        #print out['table']['XMODEL_IMAGE']
        #print RAmain
        RAmain = WCS(quas).all_world2pix(main_RA_WORLD,main_DEC_WORLD,0)[0]
        DECmain = WCS(quas).all_world2pix(main_RA_WORLD,main_DEC_WORLD,0)[1]
        #print out['table']['XMODEL_IMAGE'], out['table']['YMODEL_IMAGE']
        #print RAmain, DECmain

        for j in range(len(out['table']['XMODEL_IMAGE'])):
            #output['time'][i] = header['MJD-OBS']
            if RAmain - 30. < out['table']['XMODEL_IMAGE'][j] <  RAmain + 30. and \
            DECmain - 30. < out['table']['YMODEL_IMAGE'][j] < DECmain + 30.:
                #print 'YES'
                output = out['table']['FLUX_APER'][j]
                break
    else:
        output = float('nan')
        #print 'Done'
    return output


In [15]:
def findmag(start,compare1,RA_star,DEC_star,RA_com,DEC_com,mag_com,AIR_EXTINCTION):
    aperture1 = 0.001
    #201.5441,-8.8601
    com_flu = []
    com_AIRMASS = []
    com_EXPTIME = []
    star_flu = []
    star_AIRMASS = []
    star_EXPTIME = []
    for i in range(len(compare1)):
        #print 'Frame nr. ', i
        header = pyfits.getheader(compare1[i])
        a1 = flux(compare1[i],2*aperture1,RA_com,DEC_com)/header['EXPTIME']
        if a1 > 0:
            #print a1
            com_flu.append(a1), com_AIRMASS.append(header['AIRMASS']), com_EXPTIME.append(header['EXPTIME'])
        #plt.figure(figsize=(3,3))
        #plt.imshow(a1,cmap='hot')
        #plt.colorbar()
    for i in range(len(start)):
        print 'Frame nr. ', i, 
        header = pyfits.getheader(start[i])
        a1 = flux(start[i],2*aperture1,RA_star,DEC_star)/header['EXPTIME']
        if a1 > 0:
            #print a1
            star_flu.append(a1), star_AIRMASS.append(header['AIRMASS']), star_EXPTIME.append(header['EXPTIME'])
        #plt.figure(figsize=(3,3))
        #plt.imshow(a1,cmap='hot')
        #plt.colorbar()
    com_flu = np.array(com_flu)
    com_AIRMASS = np.array(com_AIRMASS)
    com_EXPTIME = np.array(com_EXPTIME)
    star_flu = np.array(star_flu)
    star_AIRMASS = np.array(star_AIRMASS)
    star_EXPTIME = np.array(star_EXPTIME)
    com_flu = com_flu/com_EXPTIME
    star_flu = star_flu/star_EXPTIME
    com_flux = np.mean(com_flu)
    com_AIRMASS_num = np.mean(com_AIRMASS)
    star_AIRMASS_num = np.mean(star_AIRMASS)
    mag = -(AIR_EXTINCTION)*(star_AIRMASS_num-com_AIRMASS_num) + mag_com - 2.5*np.log10(star_flu/float(com_flux))
    print mag
    return mag

In [16]:
sta1_1 = findmag(NGC3783_g,G_g,sta1_RA_WORLD,sta1_DEC_WORLD,G_RA,G_DEC,G_mag_g,k_g)
sta2_1 = findmag(NGC3783_g,G_g,sta2_RA_WORLD,sta2_DEC_WORLD,G_RA,G_DEC,G_mag_g,k_g)

sta1_2 = findmag(NGC3783_g,T_g,sta1_RA_WORLD,sta1_DEC_WORLD,T_RA,T_DEC,T_mag_g,k_g)
sta2_2 = findmag(NGC3783_g,T_g,sta2_RA_WORLD,sta2_DEC_WORLD,T_RA,T_DEC,T_mag_g,k_g)



the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]
No handlers could be found for logger "sewpy.sewpy"


Frame nr.  0 Frame nr.  1 Frame nr.  2 [ 14.46369487  14.49579454  14.57254208]


KeyboardInterrupt: 

In [ ]:
print 'g-filter'
print sta1_1
print sta1_2
print np.mean((sta1_1,sta1_2)), np.std((sta1_1,sta1_2))

print sta2_1
print sta2_2
print np.mean((sta2_1,sta2_2)), np.std((sta2_1,sta2_2))
#print np.mean(Tyc_22_1), np.std(Tyc_22_1)


In [ ]:
sta1_1 = findmag(NGC3783_r,G_r,sta1_RA_WORLD,sta1_DEC_WORLD,G_RA,G_DEC,G_mag_r,k_r)
sta2_1 = findmag(NGC3783_r,G_r,sta2_RA_WORLD,sta2_DEC_WORLD,G_RA,G_DEC,G_mag_r,k_r)

sta1_2 = findmag(NGC3783_r,T_r,sta1_RA_WORLD,sta1_DEC_WORLD,T_RA,T_DEC,T_mag_r,k_r)
sta2_2 = findmag(NGC3783_r,T_r,sta2_RA_WORLD,sta2_DEC_WORLD,T_RA,T_DEC,T_mag_r,k_r)




In [ ]:
print 'r-filter'
print sta1_1
print sta1_2
print np.mean((sta1_1,sta1_2)), np.std((sta1_1,sta1_2))

print sta2_1
print sta2_2
print np.mean((sta2_1,sta2_2)), np.std((sta2_1,sta2_2))
#print np.mean(Tyc_22_1), np.std(Tyc_22_1)


In [ ]:
sta1_1 = findmag(NGC3783_i,G_i,sta1_RA_WORLD,sta1_DEC_WORLD,G_RA,G_DEC,G_mag_i,k_i)
sta2_1 = findmag(NGC3783_i,G_i,sta2_RA_WORLD,sta2_DEC_WORLD,G_RA,G_DEC,G_mag_i,k_i)

sta1_2 = findmag(NGC3783_i,T_i,sta1_RA_WORLD,sta1_DEC_WORLD,T_RA,T_DEC,T_mag_i,k_i)
sta2_2 = findmag(NGC3783_i,T_i,sta2_RA_WORLD,sta2_DEC_WORLD,T_RA,T_DEC,T_mag_i,k_i)


In [ ]:
print 'i-filter'
print sta1_1
print sta1_2
print np.mean((sta1_1,sta1_2)), np.std((sta1_1,sta1_2))

print sta2_1
print sta2_2
print np.mean((sta2_1,sta2_2)), np.std((sta2_1,sta2_2))
#print np.mean(Tyc_22_1), np.std(Tyc_22_1)


In [17]:
sta1_1 = findmag(NGC3783_z,G_z,sta1_RA_WORLD,sta1_DEC_WORLD,G_RA,G_DEC,G_mag_z,k_z)
sta2_1 = findmag(NGC3783_z,G_z,sta2_RA_WORLD,sta2_DEC_WORLD,G_RA,G_DEC,G_mag_z,k_z)

sta1_2 = findmag(NGC3783_z,T_z,sta1_RA_WORLD,sta1_DEC_WORLD,T_RA,T_DEC,T_mag_z,k_z)
sta2_2 = findmag(NGC3783_z,T_z,sta2_RA_WORLD,sta2_DEC_WORLD,T_RA,T_DEC,T_mag_z,k_z)


Frame nr.  0 Frame nr.  1 Frame nr.  2 [ 13.1745271   13.22692305  13.33053927]
Frame nr.  0 Frame nr.  1 Frame nr.  2 [ 15.01039342  15.1673174   15.39447981]
Frame nr.  0 Frame nr.  1 Frame nr.  2 [ 13.2009855   13.25338145  13.35699767]
Frame nr.  0 Frame nr.  1 Frame nr.  2 [ 15.03685182  15.1937758   15.42093821]


In [18]:
print 'z-filter'
print sta1_1
print sta1_2
print np.mean((sta1_1,sta1_2)), np.std((sta1_1,sta1_2))

print sta2_1
print sta2_2
print np.mean((sta2_1,sta2_2)), np.std((sta2_1,sta2_2))
#print np.mean(Tyc_22_1), np.std(Tyc_22_1)


z-filter
[ 13.1745271   13.22692305  13.33053927]
[ 13.2009855   13.25338145  13.35699767]
13.2572256726 0.0661618905977
[ 15.01039342  15.1673174   15.39447981]
[ 15.03685182  15.1937758   15.42093821]
15.2039594059 0.158228160477
